In [1]:
import pandas as pd
import requests
from gnpsdata import workflow_classicnetworking

In [2]:
# Input GNPS Task
task = "24267911d87b4d96b2b43c322fc06842"

In [3]:
# Loading the library identifications

library_df = workflow_classicnetworking.get_librarymatches_dataframe(task)

/home/user/miniconda3/envs/playground/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
all_library_entries = library_df.to_dict("records")

In [5]:
CMMC_API_URL = "https://cmmc-kb.gnps2.org/api/entry"

for library_match in all_library_entries:
    #print(library_match)
    library_usi = "mzspec:GNPS:GNPS-LIBRARY:accession:{}".format(library_match["SpectrumID"])
    
    # Doing Look-Up by USI
    r = requests.get(CMMC_API_URL, params={"usi":library_usi})
    
    if len(r.json()) > 0:
        library_match["cmmc_id"] = r.json()[0]["id"]
        
    # Doing Lookup by InCHiKey
    r = requests.get(CMMC_API_URL, params={"inchikey": library_match["InChIKey"]})

    # Doing lookup by USI
    library_usi = "mzspec:GNPS:{}:accession:{}".format(library_match["SpectrumID"], library_match["SpectrumID"])
    
    if len(r.json()) > 0:
        library_match["cmmc_id"] = r.json()[0]["id"]

In [ ]:
# Writing output to file: library_matches_with_cmmc_ids.tsv

new_library_df = pd.DataFrame(all_library_entries)
new_library_df.to_csv("library_matches_with_cmmc_ids.tsv", index=False, sep="\t")